### Synchronization in Processes

### Problem in Shared Memory


In [1]:
import time
from multiprocessing import Process, Value, Lock, Pool

In [2]:
def deposit(balance):
    for i in range(1000):
        time.sleep(0.001)
        balance.value = balance.value + 1

def withdraw(balance):
    for i in range(1000):
        time.sleep(0.001)
        balance.value = balance.value - 1

def demo_shared_memory_problem():
    balance = Value('i', 2000)
    print(f'before same number of deposit and withdraw: {balance.value}')
    p1 = Process(target=deposit, args=(balance,))
    p2 = Process(target=withdraw, args=(balance,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(f'after same number of deposit and withdraw: {balance.value}')

for i in range(5):
    demo_shared_memory_problem()


before same number of deposit and withdraw: 2000
after same number of deposit and withdraw: 2078
before same number of deposit and withdraw: 2000
after same number of deposit and withdraw: 2128
before same number of deposit and withdraw: 2000
after same number of deposit and withdraw: 2116
before same number of deposit and withdraw: 2000
after same number of deposit and withdraw: 2002
before same number of deposit and withdraw: 2000
after same number of deposit and withdraw: 1908


### Multiprocessing Lock

In [3]:
def deposit_lock1(balance, lock):
    for i in range(1000):
        time.sleep(0.001)
        lock.acquire()
        balance.value = balance.value + 1
        lock.release()

def withdraw_lock1(balance, lock):
    for i in range(1000):
        time.sleep(0.001)
        lock.acquire()
        balance.value = balance.value - 1
        lock.release()

def demo_lock1():
    lock = Lock()
    balance = Value('i', 2000)
    print(f'before same number of deposit and withdraw: {balance.value}')
    p1 = Process(target=deposit_lock1, args=(balance, lock))
    p2 = Process(target=withdraw_lock1, args=(balance, lock))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(f'after same number of deposit and withdraw: {balance.value}')

for i in range(5):
    demo_lock1()

before same number of deposit and withdraw: 2000
after same number of deposit and withdraw: 2000
before same number of deposit and withdraw: 2000
after same number of deposit and withdraw: 2000
before same number of deposit and withdraw: 2000
after same number of deposit and withdraw: 2000
before same number of deposit and withdraw: 2000
after same number of deposit and withdraw: 2000
before same number of deposit and withdraw: 2000
after same number of deposit and withdraw: 2000


### Use the the Decorator Design Pattern

In [4]:
from contextlib import ContextDecorator

class SimpleLock(ContextDecorator):
    
    def __init__(self):
        self.lock = Lock()
    
    def __enter__(self):
        self.lock.acquire()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.lock.release()

Without using locks - having synchronization problem

In [11]:

balance = Value('i', 20000)

def deposit(amount):
    """
    param amount: amount of money to be deposit
    """
    global balance
    balance.value = balance.value + amount
    
def withdraw(amount):
    """
    param amount: amount of money to be withdraw
    """
    global balance
    balance.value = balance.value - amount

def deposit_task(amount, times):
    """
    param amount: amount of money to be deposit
    param times: number of transactions
    """
    global balance
    for _ in range(times):
        deposit(amount)
    
def withdraw_task(amount, times):
    """
    param amount: amount of moeny to be withdraw
    param times: number of transactions
    """
    global balance
    for _ in range(times):
        withdraw(amount)
    
def no_lock_example():
    global balance
    
    # 4 workers
    # -- 2 workers doing deposit jobs
    # ---- each worker deposit $10 500 times 
    # -- 2 workers doing withdraw jobs
    # ---- each worker withdraw $10 500 times
    # total of $10,000 deposit and $10,000 withdrawal
    # logically, expect the balance to stay the same ($20,000)
    # after all deposit and withdraw
    times = [500, 500]
    amount = [10, 10]
    
    print(f'before: {balance.value}')
    # Use two cores to deposit and two cores to withdraw
    with Pool(2) as pd, Pool(2) as pw:
        pd.starmap(deposit_task, zip(amount, times))
        pw.starmap(withdraw_task, zip(amount, times))
    print(f'after: {balance.value}')

# test the result 5 times
for _ in range(5):
    no_lock_example()
    

before: 20000
after: 20270
before: 20270
after: 20110
before: 20110
after: 20670
before: 20670
after: 21230
before: 21230
after: 21490


The example above had the same problem that a deposit transaction can happen before a withdrawal transaction completes and vice versa. Now let's change the code by adding the two lines of decorator.

In [15]:

balance = Value('i', 20000)

simple_lock = SimpleLock()

@simple_lock
def deposit(amount):
    """
    param amount: amount of money to be deposit
    """
    global balance
    balance.value = balance.value + amount

@simple_lock
def withdraw(amount):
    """
    param amount: amount of money to be withdraw
    """
    global balance
    balance.value = balance.value - amount

def deposit_task(amount, times):
    """
    param amount: amount of money to be deposit
    param times: number of transactions
    """
    global balance
    for _ in range(times):
        deposit(amount)
    
def withdraw_task(amount, times):
    """
    param amount: amount of moeny to be withdraw
    param times: number of transactions
    """
    global balance
    for _ in range(times):
        withdraw(amount)
    
def decorator_example():
    global balance
    
    # 4 workers
    # -- 2 workers doing deposit jobs
    # ---- each worker deposit $10 500 times 
    # -- 2 workers doing withdraw jobs
    # ---- each worker withdraw $10 500 times
    # total of $10,000 deposit and $10,000 withdrawal
    # logically, expect the balance to stay the same ($20,000)
    # after all deposit and withdraw
    times = [500, 500]
    amount = [10, 10]
    
    print(f'before: {balance.value}')
    # Use two cores to deposit and two cores to withdraw
    with Pool(2) as pd, Pool(2) as pw:
        pd.starmap(deposit_task, zip(amount, times))
        pw.starmap(withdraw_task, zip(amount, times))
    print(f'after: {balance.value}')

# test the result 5 times
for _ in range(5):
    decorator_example()
    

before: 20000
after: 20000
before: 20000
after: 20000
before: 20000
after: 20000
before: 20000
after: 20000
before: 20000
after: 20000


You can also use the the ContexDecorator class in a with-statement fashion.

In [16]:

balance = Value('i', 20000)

simple_lock = SimpleLock()

def deposit(amount):
    """
    param amount: amount of money to be deposit
    """
    global balance
    with simple_lock:
        balance.value = balance.value + amount

def withdraw(amount):
    """
    param amount: amount of money to be withdraw
    """
    global balance
    with simple_lock:
        balance.value = balance.value - amount

def deposit_task(amount, times):
    """
    param amount: amount of money to be deposit
    param times: number of transactions
    """
    global balance
    for _ in range(times):
        deposit(amount)
    
def withdraw_task(amount, times):
    """
    param amount: amount of moeny to be withdraw
    param times: number of transactions
    """
    global balance
    for _ in range(times):
        withdraw(amount)
    
def with_example():
    global balance
    
    # 4 workers
    # -- 2 workers doing deposit jobs
    # ---- each worker deposit $10 500 times 
    # -- 2 workers doing withdraw jobs
    # ---- each worker withdraw $10 500 times
    # total of $10,000 deposit and $10,000 withdrawal
    # logically, expect the balance to stay the same ($20,000)
    # after all deposit and withdraw
    times = [500, 500]
    amount = [10, 10]
    
    print(f'before: {balance.value}')
    # Use two cores to deposit and two cores to withdraw
    with Pool(2) as pd, Pool(2) as pw:
        pd.starmap(deposit_task, zip(amount, times))
        pw.starmap(withdraw_task, zip(amount, times))
    print(f'after: {balance.value}')

# test the result 5 times
for _ in range(5):
    with_example()
    

before: 20000
after: 20000
before: 20000
after: 20000
before: 20000
after: 20000
before: 20000
after: 20000
before: 20000
after: 20000


### Is Lock necessary in Python threading module even there's already a GIL?

YES.

The GIL only guarantees that the interpreter would not go wrong because of multithreading and that python code of the same process is executed sequentially. It does **NOT** protect your code when you have a read-modify-write function running in your thread that modifies the shared in-memory data. 

In [38]:
import threading

balance = 0
lock = threading.Lock()

def deposit_n_times(n):
    global balance
    for i in range(n):
        balance += 1

def safe_deposit_n_times(n):
    global balance
    for i in range(n):
        lock.acquire()
        balance += 1
        lock.release()

def deposit_in_x_threads(x, func, n):
    threads = [threading.Thread(target=func, args=(n,)) for _ in range(x)]
    global balance
    balance = 0
    
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print(f'total balanced: {balance} \nexpected balance: {n*x}')

for _ in range(5):
    print(25*'-')
    print('--- Unsafe Example ---')
    deposit_in_x_threads(8, deposit_n_times, 100000)
    print('--- Safe Example ---')
    deposit_in_x_threads(8, safe_deposit_n_times, 100000)

-------------------------
--- Unsafe Example ---
total balanced: 668617 
expected balance: 800000
--- Safe Example ---
total balanced: 800000 
expected balance: 800000
-------------------------
--- Unsafe Example ---
total balanced: 734083 
expected balance: 800000
--- Safe Example ---
total balanced: 800000 
expected balance: 800000
-------------------------
--- Unsafe Example ---
total balanced: 800000 
expected balance: 800000
--- Safe Example ---
total balanced: 800000 
expected balance: 800000
-------------------------
--- Unsafe Example ---
total balanced: 695088 
expected balance: 800000
--- Safe Example ---
total balanced: 800000 
expected balance: 800000
-------------------------
--- Unsafe Example ---
total balanced: 800000 
expected balance: 800000
--- Safe Example ---
total balanced: 800000 
expected balance: 800000
